In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')

In [2]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [49]:
# parameters
customer_id = '137'
formatted_attribute = 'Minimum Operating Temperature'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Timing Belt"""

regex_pattern = r"(?i)(\s[^\.]\d*\.?\d* in\w*\s?O.D.)|([^\.]\d*\.\d* in\w*\s?OAL)|([^\.]\d*\-\d*\/\d* in\w*\s?O.D.)|([^\.]\d*\.\d in\w*\s?O.D.)"
# regex_pattern = r"(?i)(\b[8][1-9] in\w* Overall Length)|(\b[9][0-9] in\w* Overall Length)|(\b1[0][0] in\w* Overall Length)"


value='%n/a%'
customer_name='%wbmason%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
#           'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [50]:
regex_pattern = r"(?i)(pore)|(strip)|()"
# regex_pattern_wreath = r"(?i)(Wreath)|()"
# regex_pattern = r"(?i)(\d+\.?\d*\s?\\\"\s?diam)|(\d+\.\d*\s?\\\"\s?diam)|(\s\.\d*\s?\\\"\s?diam)|(\d+.?\d*\/\d*\s?\\\"\s?diam)|()"
# regex_pattern_general = r"(?i)(\d+\\\")|()"
# regex_pattern_height = r"(?i)(\\\" H)|()"

In [53]:
df = query_from_file(file_name='query/Curated_lovelyskin.sql', params=params)
print(len(df))
df.head(1)

4106


attributes                snake_case_name external_id  \
0  Minimum Operating Temperature  minimum_operating_temperature     4316508   

   product_id                                prod_name  \
0  1173635390  5/16 x 1" Galvanized Hex Head Lag Screw   

                                                                                                                long_desc  \
0  HOT-DIP GALVANIZED - Provides superior corrosion protection in demanding environments. SHARP GIMLET POINT CARBON STEEL   

  curated_date  customers_id buckets  bucket_id products.family_id value  \
0   2021-08-05           137    None        NaN               None   n/a   

  resolution  
0   standard

In [54]:
df['buckets'].explode().value_counts()

Cables, Cords & Cord Management Replacement Parts & Accessories    1110
Fuses & Circuit Breakers                                            848
Fixings & Fasteners                                                 449
Cables, Cords, & Adapters                                           425
Conduit Fittings, Replacement Parts, & Accessories                  269
Electrical System                                                   193
Fastener Extractors & Applicators                                   171
Switches & Dimmers                                                  126
Cord Management                                                     105
Wire Terminals & Connectors                                          69
Heat Guns & Torches                                                   6
Tensioners                                                            1
Name: buckets, dtype: int64

In [9]:
df[(df['buckets'].astype(str)!='Fixings & Fasteners')]['value'].explode().value_counts()

n/a         230
2200 lb       3
2450 lb       1
400 lb        1
55 lb         1
2,200 lb      1
50 lb         1
Name: value, dtype: int64

In [12]:
not_fixing=df[(df['buckets'].astype(str)!='Fixings & Fasteners')]
not_fixing

attributes   snake_case_name external_id  product_id  \
14   Tensile Strength  tensile_strength       82366  1173640095   
15   Tensile Strength  tensile_strength       82365  1173640752   
16   Tensile Strength  tensile_strength       82361  1173640759   
17   Tensile Strength  tensile_strength       82368  1173640766   
18   Tensile Strength  tensile_strength      821073  1173641214   
19   Tensile Strength  tensile_strength       82298  1173641283   
20   Tensile Strength  tensile_strength       82249  1173641284   
21   Tensile Strength  tensile_strength       82297  1173641285   
22   Tensile Strength  tensile_strength       82245  1173641286   
23   Tensile Strength  tensile_strength       82296  1173641287   
24   Tensile Strength  tensile_strength       82246  1173641288   
25   Tensile Strength  tensile_strength       82294  1173641289   
26   Tensile Strength  tensile_strength       82295  1173641290   
27   Tensile Strength  tensile_strength       82293  1173641292   
28   Tensile Strength  tensile_strength       82247  1173641293   
29   Tensile Strength  tensile_strength       82248  1173641294   
30   Tensile Strength  tensile_strength       82981  1173641295   
31   Tensile Strength  tensile_strength       82990  1173641296   
32   Tensile Strength  tensile_strength       82998  1173641297   
33   Tensile Strength  tensile_strength       82980  1173641298   
34   Tensile Strength  tensile_strength       82991  1173641299   
35   Tensile Strength  tensile_strength       82992  1173641300   
36   Tensile Strength  tensile_strength       82993  1173641301   
37   Tensile Strength  tensile_strength       82996  1173641302   
38   Tensile Strength  tensile_strength       82997  1173641303   
39   Tensile Strength  tensile_strength       82994  1173641304   
40   Tensile Strength  tensile_strength       82995  1173641305   
41   Tensile Strength  tensile_strength       82999  1173641306   
42   Tensile Strength  tensile_strength      841174  1173641308   
43   Tensile Strength  tensile_strength       82200  1173641309   
44   Tensile Strength  tensile_strength       82215  1173641310   
45   Tensile Strength  tensile_strength       82222  1173641311   
46   Tensile Strength  tensile_strength       82223  1173641312   
47   Tensile Strength  tensile_strength       82203  1173641313   
48   Tensile Strength  tensile_strength      822404  1173641314   
49   Tensile Strength  tensile_strength       82201  1173641315   
50   Tensile Strength  tensile_strength       82216  1173641316   
51   Tensile Strength  tensile_strength       82208  1173641317   
52   Tensile Strength  tensile_strength       82205  1173641318   
53   Tensile Strength  tensile_strength       82202  1173641319   
54   Tensile Strength  tensile_strength       82211  1173641320   
55   Tensile Strength  tensile_strength       82204  1173641321   
56   Tensile Strength  tensile_strength       82214  1173641323   
57   Tensile Strength  tensile_strength       82209  1173641324   
58   Tensile Strength  tensile_strength       82212  1173641325   
59   Tensile Strength  tensile_strength       82210  1173641326   
60   Tensile Strength  tensile_strength       82207  1173641327   
61   Tensile Strength  tensile_strength       82221  1173641328   
62   Tensile Strength  tensile_strength       82217  1173641329   
63   Tensile Strength  tensile_strength       82213  1173641331   
64   Tensile Strength  tensile_strength       82206  1173641332   
65   Tensile Strength  tensile_strength       82218  1173641333   
66   Tensile Strength  tensile_strength       82220  1173641335   
67   Tensile Strength  tensile_strength      821185  1173641336   
68   Tensile Strength  tensile_strength      825480  1173641337   
69   Tensile Strength  tensile_strength      825481  1173641338   
70   Tensile Strength  tensile_strength      825482  1173641339   
71   Tensile Strength  tensile_strength      825483  1173641340   
72   Tensile Strength  tensile_strength      825484  11736

In [14]:
not_fixings=not_fixing[['external_id','value']]
not_fixings#['value'].explode().value_counts()

external_id     value
14        82366       n/a
15        82365       n/a
16        82361       n/a
17        82368       n/a
18       821073       n/a
19        82298       n/a
20        82249       n/a
21        82297       n/a
22        82245       n/a
23        82296       n/a
24        82246       n/a
25        82294       n/a
26        82295       n/a
27        82293       n/a
28        82247       n/a
29        82248       n/a
30        82981       n/a
31        82990       n/a
32        82998       n/a
33        82980       n/a
34        82991       n/a
35        82992       n/a
36        82993       n/a
37        82996       n/a
38        82997       n/a
39        82994       n/a
40        82995       n/a
41        82999       n/a
42       841174       n/a
43        82200       n/a
44        82215       n/a
45        82222       n/a
46        82223       n/a
47        82203       n/a
48       822404       n/a
49        82201       n/a
50        82216       n/a
51        82208       n/a
52        82205       n/a
53        82202       n/a
54        82211       n/a
55        82204       n/a
56        82214       n/a
57        82209       n/a
58        82212       n/a
59        82210       n/a
60        82207       n/a
61        82221       n/a
62        82217       n/a
63        82213       n/a
64        82206       n/a
65        82218       n/a
66        82220       n/a
67       821185       n/a
68       825480       n/a
69       825481       n/a
70       825482       n/a
71       825483       n/a
72       825484       n/a
73       825485       n/a
74       825486       n/a
75       825487       n/a
76       825488       n/a
77       825489       n/a
78       825490       n/a
79       821186       n/a
80       821181       n/a
81        82364       n/a
82       821184       n/a
83       821180       n/a
84       821178       n/a
85       821182       n/a
86       821176       n/a
87       821177       n/a
88       821183       n/a
89       821179       n/a
90       821175       n/a
91       821350       n/a
92       821355       n/a
93       823490       n/a
94       821351       n/a
95       821352       n/a
96       821353       n/a
97       821354       n/a
98       827162       n/a
99       827163       n/a
100      827164       n/a
101      827165       n/a
102      827169       n/a
103      827171       n/a
104      827170       n/a
105      827177       n/a
106      827168       n/a
107       82362       n/a
108      827176       n/a
109      827167       n/a
110      827174       n/a
111      827166       n/a
112      827178       n/a
113      827175       n/a
114       82367       n/a
115      827180       n/a
116      827173       n/a
117      827172       n/a
118      827179       n/a
119      827181       n/a
120      822400       n/a
121      822403       n/a
122      822405       n/a
123      822406       n/a
124      822407       n/a
125      822408       n/a
126      822415       n/a
127      822410       n/a
128      822412       n/a
129      822409       n/a
130      822411       n/a
131       82360       n/a
132       82363       n/a
133      822480       n/a
134      823480       n/a
135       82350       n/a
136       82436       n/a
137      822483       n/a
138      822484       n/a
139      822485       n/a
140      823486       n/a
141      823487       n/a
142      823482       n/a
143       82351       n/a
144      823483       n/a
145      823485       n/a
146       82354       n/a
147      823484       n/a
148       82355       n/a
149       82352       n/a
150       82353       n/a
151       82356       n/a
152       82381       n/a
153      821090       n/a
154      821091       n/a
155      821043       n/a
156      821076       n/a
157      821044       n/a
158      821045       n/a
159      821068       n/a
160      821070       n/a
161      821041       n/a
162      821069       n/a
163      821077       n/a
164      821046       n/a
165      821047       n/a
166      821042       n/a


In [23]:
not_fixings['Q:tensile_strength']=''
not_fixings['Q:tensile_strength_lbs']=not_fixings['value']
not_fixingz=not_fixings[['external_id','Q:tensile_strength','Q:tensile_strength_lbs']]
not_fixingz

external_id Q:tensile_strength Q:tensile_strength_lbs
14        82366                                       n/a
15        82365                                       n/a
16        82361                                       n/a
17        82368                                       n/a
18       821073                                       n/a
19        82298                                       n/a
20        82249                                       n/a
21        82297                                       n/a
22        82245                                       n/a
23        82296                                       n/a
24        82246                                       n/a
25        82294                                       n/a
26        82295                                       n/a
27        82293                                       n/a
28        82247                                       n/a
29        82248                                       n/a
30        82981                                       n/a
31        82990                                       n/a
32        82998                                       n/a
33        82980                                       n/a
34        82991                                       n/a
35        82992                                       n/a
36        82993                                       n/a
37        82996                                       n/a
38        82997                                       n/a
39        82994                                       n/a
40        82995                                       n/a
41        82999                                       n/a
42       841174                                       n/a
43        82200                                       n/a
44        82215                                       n/a
45        82222                                       n/a
46        82223                                       n/a
47        82203                                       n/a
48       822404                                       n/a
49        82201                                       n/a
50        82216                                       n/a
51        82208                                       n/a
52        82205                                       n/a
53        82202                                       n/a
54        82211                                       n/a
55        82204                                       n/a
56        82214                                       n/a
57        82209                                       n/a
58        82212                                       n/a
59        82210                                       n/a
60        82207                                       n/a
61        82221                                       n/a
62        82217                                       n/a
63        82213                                       n/a
64        82206                                       n/a
65        82218                                       n/a
66        82220                                       n/a
67       821185                                       n/a
68       825480                                       n/a
69       825481                                       n/a
70       825482                                       n/a
71       825483                                       n/a
72       825484                                       n/a
73       825485                                       n/a
74       825486                                       n/a
75       825487                                       n/a
76       825488                                       n/a
77       825489                                       n/a
78       825490                                       n/a
79       821186                                       n/a
80       821181                                       n/a
81        82364                                       n/a
82

In [18]:
# df = query_from_file(file_name='./query/Curated_SQL_Query.sql', params=params)
# df

In [9]:
df['app_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [ ]:
# df[df['app_matches'].astype(str)!='[]']

In [ ]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
diam=df[df['matches'].astype(str)!='[]']
len(df[df['matches'].astype(str)!='[]'])

In [ ]:
diam=diam[diam['buckets'].astype(str)!='Lamps']
len(diam)

In [ ]:
matches=diam

In [ ]:
matches[curation_col]=matches['matches']

In [ ]:
df['inch_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_inch, x))
inch=df[df['inch_matches'].astype(str)!='[]']
len(inch)

In [ ]:
df['height_matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
height=df[df['height_matches'].astype(str)!='[]']
len(height)

In [ ]:
gen=df[(df['general_matches'].astype(str)!='[]')&(df['wreath_matches'].astype(str)!='[]')&(df['height_matches'].astype(str)=='[]')]
len(gen)

In [ ]:
na_upload=gen
na_upload[curation_col]=r'n/a'

In [ ]:
# def split_col(data):
#     data['matches']=data['matches'].str.get(0)
# #     data=data.dropna()
#     data['matches']=data['matches'].apply(lambda x:x.lower())
# #     data[['length', 'width']]=data['matches'].str.split(" x ",expand=True,)

In [ ]:
# def drop_characters(data):
#     data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
#     data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
#     data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
#     data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
#     data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
#     data['width']=data['width'].apply(lambda quote: quote.replace('-',''))

In [ ]:
# def replace_none(data):
#     data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
#     data['length']=data['length'].apply(lambda x: 0 if x == '' else x)
#     data['width']=data['width'].apply(lambda x: 0 if x == 'None' else x)
#     data['length']=data['length'].apply(lambda x: 0 if x == 'None' else x)

In [ ]:
# def add_quotes(data):
#     data['length']=data['length'].apply(lambda x: str(x) + "'")
#     data['width']=data['width'].apply(lambda x: str(x) + "'")

In [ ]:
# def min_max(data):
#     data['length']=data['length'].astype(float)
#     data['width']=data['width'].astype(float)
#     data['Max'] = data[['length', 'width']].values.max(1)
#     data['Min'] = data[['length', 'width']].values.min(1)

In [ ]:
# def remove_fractions(numbers):
#     if numbers.find('-') > -1:
#         numbers = literal_eval(numbers)
#         inte, fract = numbers.split(' ')
#         nom, den = fract.split('/')
#         tmp = float(nom)/float(den)
#         num = int(inte) + tmp
#         return str(num)
#     else:
#         return numbers

In [ ]:
# # lamps[['length', 'width']]=lamps['matches'].str.split(",",expand=True,)
# lamps['matches']=lamps['matches'].astype(str)

In [ ]:
# if lamps['matches'].find(',')>-1:
#     lamps[['length', 'width']]=lamps['matches'].apply(lambda x: x.split(','))

In [ ]:
# split_col(df) #might throw an error and have to drop nane

In [ ]:
# df['width']=df['width'].astype(str).apply(lambda x: 0 if x == 'nan' else x)

In [ ]:
# df['length']=df['length'].astype(str)
# df['width']=df['width'].astype(str)
# df['length']=df['length'].apply(lambda quote: quote.replace('-', ' '))
# df['width']=df['width'].apply(lambda quote: quote.replace('-', ' '))
# df['length']=df['length'].apply(lambda quote: quote.replace('inch', ''))
# df['width']=df['width'].apply(lambda quote: quote.replace('inch', ''))
# df

In [ ]:
# df['length']=df['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
# df['width']=df['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
# df

In [ ]:
# a=df['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# df['length']=c
# df

In [ ]:
# a=df['width'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# df['width']=c
# df

In [ ]:
# x=df
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# x['new'] = c
# x

In [ ]:
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)

In [ ]:
# remove_fractions(df['length'])

In [ ]:
# x=df
# a=eval(x['length'][3:4].astype(str))
# # numbers = eval(a)

In [ ]:
# float(sum(Fraction(s) for s in a))

In [ ]:
# from fractions import Fraction
# float(sum(Fraction(s) for s in '  12 1/2'.split()))

In [ ]:
# x=df
# x['length'] = x['length'].apply(lambda quote: quote.replace('-', ''))
# x['length'] = x['length'].apply(lambda quote: quote.replace('inch', ''))
# # x['length'] = x['length'].apply(lambda x: x.str)
# x

In [ ]:
# x[['first', 'second']]=x['length'].str.split("/",expand=True,)
# x[['third', 'fourth']= x['first'].split()
# result = x['third'] + (float(x['first'])/float(x['second']))
# result

In [ ]:
# num,den = s.split( '/' )
# wh, num = num.split()
# result = wh + (float(num)/float(den))

In [ ]:
# a=x['length'][3:4]
# a.str.split(' ')
# b= [[" ".join(tag.split(" ")[:2]) , " ".join(tag.split(" ")[2:])] for tag in a][0]
# b

In [ ]:
# c=b[1]
# c.split('/')

In [ ]:
# b[0]
# b

In [ ]:
# a.apply(lambda x: sum(Fraction(x.split('/'))))

In [ ]:
# float(sum(Fraction(s) for s in x['length'].split()))
# x['length']=x['length'].apply(lambda a: float(sum(Fraction(a.split()))))

In [ ]:
# for i in x['length']:
#     if x['length'][i].find('/') > -1:
#         float(sum(Fraction(s) for s in x['length'][i].split()))
#     else:
#         x['length']

In [ ]:
# float(sum(Fraction(s) for s in x['length'][i].split()))
# .apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))

In [ ]:
# def fraction_combo(numbers):
#     if numbers.find('/') > -1:
#         numbers.apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))
#         #numbers = literal_eval(numbers)
#         #inte, fract = numbers.split(' ')
#         #nom, den = fract.split('/')
#         #tmp = float(nom)/float(den)
#         #num = int(inte) + tmp
#         return str(numbers)
#     else:
#         return numbers

In [ ]:
# a=x['length']

In [ ]:
# remove_fractions(df)

In [ ]:
# x['length'] = x['length'].apply(lambda x: remove_fractions(x))
# x.head

In [ ]:
# x['length'] = x['length'].apply(lambda x: x.split(' '))
# x.head()

In [ ]:
min_max(df) #error base 10 when there is only a value for max and not min

In [ ]:
# # range function builder
# string = range_string.replace(metric, '')
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

In [ ]:
# matches=df
# matches['Max']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
# matches

In [ ]:
# matches[curation_col] = matches['Max'].astype(str)+' in'
# # matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [24]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/fixed - {attribute} - Wipe.csv',index=False)

In [25]:
looks_good('Kimball Midwest', attribute, df, not_fixingz)